In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
# Just for theme
plt.rcParams['figure.dpi']= 200
# from jupyterthemes import jtplot
# jtplot.style(theme='monokai')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATADIR = './drive/MyDrive/Codes/data/image_data/test'
CATEGORIES = ['neutral','fire','smoke']

In [ ]:
# Đường dẫn đến thư mục chứa hình ảnh cần xóa
path_to_delete = os.path.join(DATADIR, CATEGORIES[1])

# Lặp qua các tệp trong thư mục
for i in range(1, 114):
    filename = f"{i}.jpg"
    filepath = os.path.join(path_to_delete, filename)

    # Kiểm tra xem tệp tồn tại không trước khi xóa
    if os.path.exists(filepath):
        os.remove(filepath)
        print(f"Đã xóa {filename}")
    else:
        print(f"{filename} không tồn tại")

In [4]:
IMG_SIZE = 64
def create_training_data():
    training_data = []
    for category in CATEGORIES:

        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

    return training_data

In [5]:
training_data = create_training_data()

100%|██████████| 181/181 [00:03<00:00, 57.64it/s] 


In [6]:
import random
test_image_num=len(training_data)
print(len(training_data))
random.shuffle(training_data)
test_labels=np.zeros((test_image_num,1))

c=0
for sample in training_data:
    test_labels[c]=(sample[1])
    c+=1
print(c)
actual_labels=(test_labels.reshape(test_image_num,))
actual_labels.astype(int)

382
382


array([2, 2, 0, 2, 0, 2, 1, 1, 1, 1, 2, 1, 0, 0, 2, 2, 0, 0, 2, 2, 1, 1,
       2, 0, 1, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2,
       1, 1, 2, 0, 2, 0, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 0, 2, 1, 0, 2, 2,
       1, 0, 2, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 2, 1, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 2, 2, 2, 0, 2, 0, 1, 2, 1,
       0, 0, 2, 0, 2, 0, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2, 1, 2, 0, 1, 2, 2,
       2, 2, 1, 0, 0, 2, 0, 1, 2, 2, 2, 1, 1, 0, 1, 2, 2, 1, 1, 1, 2, 2,
       0, 2, 0, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 0, 2, 1, 1, 2, 2, 0, 0,
       1, 1, 0, 0, 2, 0, 0, 1, 2, 2, 1, 0, 0, 2, 2, 2, 0, 0, 0, 1, 0, 2,
       0, 2, 2, 2, 0, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 2, 1, 2, 2, 1, 0, 1,
       1, 2, 2, 1, 2, 2, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2,
       2, 1, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 0, 0, 2, 2, 2, 1, 2, 0, 1,
       2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2,
       1, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0, 0, 2, 2, 2,

In [7]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]

(64, 64, 3)

In [8]:
from tensorflow.keras.models import load_model
model=load_model('./drive/MyDrive/Codes/trained_model_v2.h5')

In [11]:
predicted_labels=np.argmax(model.predict(X), axis=-1)
predicted_labels=(predicted_labels.reshape(test_image_num,))
predicted_labels.astype(int)

12/12 [==============================] - 1s 72ms/step
[2. 2. 0. 2. 0. 2. 1. 1. 1. 1. 2. 1. 0. 0. 2. 2. 0. 0. 2. 2. 1. 1. 2. 0.
 1. 2. 2. 2. 2. 1. 1. 2. 0. 2. 2. 0. 1. 2. 2. 0. 2. 2. 2. 2. 1. 1. 2. 0.
 2. 0. 2. 2. 2. 1. 1. 1. 1. 2. 2. 1. 0. 2. 1. 0. 2. 2. 1. 0. 2. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 1. 2. 1. 2. 2. 0. 2. 2. 0. 0. 1. 2. 0. 1. 1. 0. 1. 2.
 1. 1. 0. 2. 2. 2. 2. 2. 0. 2. 0. 1. 2. 1. 0. 0. 2. 0. 2. 0. 2. 2. 2. 1.
 0. 2. 2. 1. 1. 2. 1. 2. 0. 1. 2. 2. 2. 2. 1. 0. 0. 2. 0. 1. 2. 2. 2. 1.
 1. 0. 1. 2. 2. 1. 1. 1. 2. 2. 0. 2. 0. 2. 1. 1. 2. 1. 2. 2. 1. 2. 1. 2.
 0. 2. 1. 1. 2. 2. 0. 0. 1. 1. 0. 0. 2. 0. 0. 1. 2. 2. 1. 0. 0. 2. 2. 2.
 0. 0. 0. 1. 0. 2. 0. 2. 2. 2. 0. 2. 0. 2. 0. 1. 0. 1. 1. 0. 0. 2. 1. 2.
 2. 1. 0. 1. 1. 2. 2. 1. 2. 2. 2. 2. 2. 0. 1. 0. 2. 2. 2. 2. 1. 2. 0. 2.
 2. 2. 2. 1. 2. 2. 2. 1. 2. 1. 0. 1. 2. 2. 1. 0. 0. 2. 2. 2. 1. 2. 0. 1.
 2. 2. 2. 2. 2. 2. 0. 2. 0. 0. 2. 2. 2. 0. 2. 1. 2. 2. 0. 2. 0. 2. 1. 0.
 0. 0. 0. 1. 0. 2. 2. 0. 0. 0. 2. 2. 2. 0. 1. 2. 1. 2. 0. 1. 2. 2. 2. 

In [12]:
from sklearn.metrics import accuracy_score

print(accuracy_score(actual_labels,predicted_labels))

0.7617801047120419


In [14]:
tp=cm[0][0]
fn=cm[0][1]
fp=cm[1][0]
tn=cm[1][1]
print("tp"+' '+str(tp))
print("fn"+' '+str(fn))
print("fp"+' '+str(fp))
print("tn"+' '+str(tn))

tp 74
fn 16
fp 4
tn 90


In [15]:
Recall=tp/(tp+fn)
Precision=tp/(tp+fp)
f_measure= 2*((Precision*Recall)/(Precision+Recall))
print(Precision, Recall, f_measure)

0.9487179487179487 0.8222222222222222 0.8809523809523809


In [ ]:
# 0.11428571428571428 0.07239819004524888 0.0886426592797784